# SAS Viya Interactive Matrix Language

This notebook shows how to use the new IML Action Set on the SAS Viya Platform.

# Load Packages

In [1]:
import swat
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import time
swat.options.cas.print_messages = True

# Connect to CAS

In [2]:
conn = swat.CAS("server", 8777, "student", "Metadata0", protocol="http")

# Load the IML Action Set

In [3]:
conn.loadActionSet('iml')
actions = conn.builtins.help(actionSet='iml')

NOTE: Added action set 'iml'.
NOTE: Information for action set 'iml':
NOTE:    iml
NOTE:       iml - Submits SAS/IML programs in CAS


# Call the IML Action

In [4]:
conn.iml.iml(code=
'''

msg1 = 'Hello, World!'; 
print msg1;
x = 2+2;
print x;

'''
)

,COL1
0,"Hello, World!"
,COL1
0,4.0


In [5]:
tmp = conn.iml.iml(code=
'''

msg1 = 'Hello, World!'; 
print msg1;
x = 2+2;
print x;

'''
)

display(list(tmp))
display(tmp['Print1.msg1'])
display(tmp['Print2.x'])

x = tmp['Print2.x']['COL1'][0]
x

['Print1.msg1', 'Print2.x']

,COL1
0,"Hello, World!"


,COL1
0,4.0


4.0

In [6]:
conn.iml.iml(code=
'''

call randseed(27606);
n = 200;
beta0 = 5;
beta1 = 2;
xvals = j(n,1,0);
call randgen(xvals,"Uniform");
xvals = xvals*20;
error = j(n,1,0);
call randgen(error,"Normal",0,5);
y = beta0 + beta1*xvals + error;
    
x = j(n,1,1)||xvals;
xpx = x`*x;
    
call svd(u,q,v,xpx);
xpxSVD = u*diag(q)*v`;
    
qInv = 1/q;
qInvDiag = diag(qInv);
    
betaHat = (v*qInvDiag*u`)*(x`*y);
print betaHat;

'''
)

,COL1
0,4.299162
1,2.019846


[Procedure vs Action Differences](https://documentation.sas.com/?docsetId=casactiml&docsetTarget=casactiml_iml_details02.htm&docsetVersion=8.5&locale=en)

# Read and Write CAS Tables

In [7]:
castbl = conn.read_csv("D:/Workshop/winsas/VOSI/hmeq.csv", casout = dict(name="hmeq", replace=True))
conn.table.fetch(table='hmeq', to=3)

NOTE: Cloud Analytic Services made the uploaded file available as table HMEQ in caslib CASUSER(student).
NOTE: The table HMEQ has been created in caslib CASUSER(student) from binary data uploaded to Cloud Analytic Services.


,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1.0,1100.0,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1.0,1300.0,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1.0,1500.0,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN


In [8]:
conn.iml.iml(code=
'''
    
mymat = MatrixCreateFromCAS('hmeq');
bad = mymat[,1];
bad_mean = bad[:];
print bad_mean;
    
colmeans = mymat[:,];
varnames = {'BAD' 'LOAN' 'MORTDUE' 'VALUE' 'YOJ' 'DEROG' 'DELINQ' 'CLAGE' 'NINQ' 'CLNO' 'DEBTINC'};
call MatrixWriteToCAS(colmeans,'', 'hmeq_means', varNames); 
    
'''
)

,COL1
0,0.199497


In [9]:
conn.table.fetch(table='hmeq_means')

,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,_ROWID_
0,0.199497,18607.969799,73760.8172,101776.048741,8.922268,0.25457,0.449442,179.766275,1.186055,21.296096,33.779915,1.0


# Run a Simulation in Parallel

[nPerThread Function](https://documentation.sas.com/?docsetId=casactiml&docsetTarget=casactiml_langref_sect062.htm&docsetVersion=8.5&locale=en)

The primary use of the NPERTHREAD function is to evenly divide independent repetitions among tasks that are running in a parallel.

In [10]:
threads = conn.table.tableDetails(
    level="node",
    caslib="casuser",
    name="hmeq"
)['TableDetails']['Blocks'][0]

threads

16

In [11]:
conn.iml.iml(code=
'''

num = nPerThread(160); /*nTotal = the total number of repetitions that should be completed over all available threads.*/
print num;
    
'''
)

,COL1
0,10.0


Sampling Distribution for a Uniform(0,1) distribution with n=100 samples.

$X \sim U(a=0,b=1) \quad E(X) = \frac{b-a}{2} = \frac{1}{2} \quad VAR(X) = \frac{(b-a)^2}{12} = \frac{1}{12}$

$\bar{X} \sim N \bigg( \mu = E(X)=\frac{1}{2}, \enspace \sigma=\sqrt{\frac{\sigma_X^2}{n}}= \sqrt{\frac{1/12}{100}}=\sqrt{\frac{1}{1200}}=0.02887 \bigg)$

In [12]:
#######################
### Single Threaded ###
#######################

start = time.time()
Uniform_SD_ST = conn.iml.iml(code=
"""

/* Create function to simulate Uniform(0,1) samples */
start SimMeanUniform(params);
    n_obs = params$'n_obs';
    n_samples = params$'n_samples';

    /* Generate n_obs x n_samples matrix. Each column is sample */
    X = j(n_obs, n_samples);
    call randgen(X, 'Uniform');
     
    /* Return mean of each column in row vector */
    return mean(X);                   
finish;


/* Specify arguments in a list for the mapping function */
mylist = [#'n_obs'   = 100,     /* sample size */
          #'n_samples' = 1E6 ]; /* total number of samples */
   
stat = SimMeanUniform(mylist);
stat = stat`;
alpha = 0.05;
tot_samples = nrow(stat);              
   
/* Sumamrize Sampling Distribution */
MCEst = mean(stat); 
SE = std(stat);
call qntl(CI, stat, alpha/2 || 1-alpha/2);
results = tot_samples || MCEst || SE || CI`; /* combine results for printing */
print results[format=8.4 L='Monte Carlo Estimates and 95% CI' c={'NumSamples' 'MCEst' 'StdErr' 'LowerCL' 'UpperCL'}];
   
"""
)

Uniform_SD_ST['Print1.results'].columns = ["NumSamples","Mean","StdErr","LowerCL","UpperCL"]
print(Uniform_SD_ST['Print1.results'])

end = time.time()
sim_single_time = end-start
print("Total Time Single Thread =", round(sim_single_time,2), "Seconds")

NOTE: Module SIMMEANUNIFORM defined.
Monte Carlo Estimates and 95% CI

   NumSamples      Mean    StdErr   LowerCL   UpperCL
0   1000000.0  0.499994  0.028882  0.443286  0.556645
Total Time Single Thread = 2.21 Seconds


In [13]:
###################
### Distributed ###
###################

start = time.time()
Uniform_SD_MT = conn.iml.iml(nthreads=16, code=
"""

/* Create function to simulate Uniform(0,1) samples */
start SimMeanUniform(params);
    n_obs = params$'n_obs';
      
    /* Find number of samples per thread */
    n_samples_per_thread = nPerThread(params$'n_samples');

    /* Generate n_obs x n_samples matrix. Each column is sample */
    X = j(n_obs, n_samples_per_thread);
    call randgen(X, 'Uniform');
     
    /* Return mean of each column in row vector */
    return mean(X);                   
finish;

/* Specify arguments in a list for the mapping function */
mylist = [#'n_obs'   = 100,     /* sample size */
          #'n_samples' = 1E6 ]; /* total number of samples */

stat = MapReduce(mylist, 'SimMeanUniform', '_HCONCAT');
stat = stat`;
alpha = 0.05;
tot_samples = nrow(stat);              
   
/* Sumamrize Sampling Distribution */
MCEst = mean(stat); 
SE = std(stat);
call qntl(CI, stat, alpha/2 || 1-alpha/2);
results = tot_samples || MCEst || SE || CI`; /* combine results for printing */
print results[format=8.4 L='Monte Carlo Estimates and 95% CI' c={'NumSamples' 'MCEst' 'StdErr' 'LowerCL' 'UpperCL'}];
   
"""
)

Uniform_SD_MT['Print1.results'].columns = ["NumSamples","Mean","StdErr","LowerCL","UpperCL"]
print(Uniform_SD_MT['Print1.results'])

end = time.time()
sim_dist_time = end-start
print("Total Time Distributed =", round(sim_dist_time,2), "Seconds")
print("Distributed Simulation is", round(sim_single_time/sim_dist_time,2),"times faster")

NOTE: Module SIMMEANUNIFORM defined.
Monte Carlo Estimates and 95% CI

   NumSamples      Mean    StdErr   LowerCL   UpperCL
0   1000000.0  0.499985  0.028897  0.443387  0.556634
Total Time Distributed = 0.67 Seconds
Distributed Simulation is 3.28 times faster


# Run Tasks in Parallel

1. The Monty Hall Game Show Problem
2. Sampling Distribution Simple Linear Regression
3. Eigen Values for Toeplitz Matrix

In [14]:
run_tasks = conn.iml.iml(nthreads=3, code=
"""

/****************************************/
/*** The Monty Hall Game Show Problem ***/
/****************************************/

start MontyHall(Ngames);

    call randseed(802);

    do iteration=1 to Ngames;
        doors = {1 2 3};
        carDoor=sample(doors,1);

        *Pick door for Monty Hall to open;
        if carDoor=1 then openDoor=randfun(1,"Bernoulli",.5) + 2;
        else if carDoor=2 then openDoor=3;
        else if carDoor=3 then openDoor=2;

        *Determine door for switching strategy;
        if openDoor=2 then switchDoor=3;
        else if openDoor=3 then switchDoor=2;

        *Determine which strategy wins;
        if carDoor=1 then stayWin=1;
        else stayWin=0;
        if carDoor=switchDoor then switchWin=1;
        else switchWin=0;

        *Collect results to a single matrix;
        results=results // (stayWin || switchWin);
    end;

    return(results[:,]);
    
finish;

/******************************************************/
/*** Sampling Distribution Simple Linear Regression ***/
/******************************************************/

start simpleRegFunc(SLR);

    n = SLR$1;
    beta0 = SLR$2;
    beta1 = SLR$3;
    reps = SLR$4;

    beta0vec = j(reps,1,.);
    beta1vec = j(reps,1,.);
    call randseed(27606);

    do i=1 to reps;
        xvals = randfun(n,"Uniform");
        xvals = xvals*20;
        error = randfun(n,"Normal",0,5);
        y = beta0 + beta1*xvals + error;
        x = j(n,1,1)||xvals;
        betaHat = inv(x`*x)*(x`*y);

        beta0vec[i] = betaHat[1];
        beta1vec[i] = betaHat[2];
    end;

    mean0 = mean(beta0vec);
    sd0 = std(beta0vec);

    mean1 = mean(beta1vec);
    sd1 = std(beta1vec);

    out0 = mean0//sd0;
    out1 = mean1//sd1;

    return(out0||out1);
    
finish;

/****************************************/
/*** Eigen Values for Toeplitz Matrix ***/
/****************************************/

start GetEigenValues(matSize);

    mat = toeplitz(matSize:1);
    toe_eigs = eigval(mat);
    top_toe_eigs = toe_eigs[1:3,];
    return top_toe_eigs;
    
finish;

/****************************************/
/*** Distribute Tasks and Get Results ***/
/****************************************/

Tasks = {'MontyHall' 'simpleRegFunc' 'GetEigenvalues'};
Args = [1000, [20,5,2,1000], 250];
opt = {1,     /* distribute tasks to threads on controller */
       1};    /* display information about the tasks */
results = ParTasks(Tasks, Args, opt);
print results;

"""
)

print(list(run_tasks.keys()))
print(run_tasks['Print1.ParallelTaskInfo'])
run_tasks['Print2.results.results$1'].columns = ["Stay Win","Switch Win"]
run_tasks['Print2.results.results$2'].columns = ["Mean","Standard Error"]
run_tasks['Print2.results.results$3'].columns = ["Top 3 Eigen Values"]
run_tasks

NOTE: Module MONTYHALL defined.
NOTE: Module SIMPLEREGFUNC defined.
NOTE: Module GETEIGENVALUES defined.
['Print1.ParallelTaskInfo', 'Print2.results.results$1', 'Print2.results.results$2', 'Print2.results.results$3']
Parallel Tasks Information

   TaskNumber            Task  Node  Thread      Time    OHTime
0           1       MontyHall     0       1  0.004429  0.000003
1           2   simpleRegFunc     0       2  0.020126  0.005749
2           3  GetEigenvalues     0       3  0.010003  0.005728


[Print1.ParallelTaskInfo]

 Parallel Tasks Information
 
    TaskNumber            Task  Node  Thread      Time    OHTime
 0           1       MontyHall     0       1  0.004429  0.000003
 1           2   simpleRegFunc     0       2  0.020126  0.005749
 2           3  GetEigenvalues     0       3  0.010003  0.005728

[Print2.results.results$1]

 results$1
 
    Stay Win  Switch Win
 0     0.337       0.663

[Print2.results.results$2]

 results$2
 
        Mean  Standard Error
 0  5.044306        1.993124
 1  2.358394        0.204477

[Print2.results.results$3]

 results$3
 
    Top 3 Eigen Values
 0        42220.097279
 1        12665.314623
 2         2663.196307

+ Elapsed: 0.0963s, user: 0.133s, sys: 0.0118s, mem: 16.8mb

# End the Session

In [15]:
conn.session.endSession()

+ Elapsed: 0.000209s, user: 0.000137s, sys: 4.6e-05s, mem: 0.222mb